In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark import SparkConf,SparkContext
sc = SparkContext()

In [5]:
data=sc.textFile('books.csv')

In [6]:
data.top(1)

['id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url']

In [8]:
columns=data.first().split(',')
columns

In [ ]:
# Spark Data frame API (SQL)
# To work with this we should use SparkSession

In [11]:
import pyspark as ps
spark=ps.sql.SparkSession.builder.getOrCreate()

In [12]:
type(spark)

pyspark.sql.session.SparkSession

In [14]:
# Importing books data to SQL data frame using SparkSession
booksdf=spark.read.csv('books.csv',header=True,inferSchema=True)

# Lets see the schema of the frame
booksdf.printSchema()

root
 |-- id: integer (nullable = true)
 |-- book_id: integer (nullable = true)
 |-- best_book_id: integer (nullable = true)
 |-- work_id: integer (nullable = true)
 |-- books_count: integer (nullable = true)
 |-- isbn: string (nullable = true)
 |-- isbn13: double (nullable = true)
 |-- authors: string (nullable = true)
 |-- original_publication_year: double (nullable = true)
 |-- original_title: string (nullable = true)
 |-- title: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- average_rating: string (nullable = true)
 |-- ratings_count: string (nullable = true)
 |-- work_ratings_count: string (nullable = true)
 |-- work_text_reviews_count: string (nullable = true)
 |-- ratings_1: double (nullable = true)
 |-- ratings_2: integer (nullable = true)
 |-- ratings_3: integer (nullable = true)
 |-- ratings_4: integer (nullable = true)
 |-- ratings_5: integer (nullable = true)
 |-- image_url: string (nullable = true)
 |-- small_image_url: string (nullable = true)


In [15]:
len(booksdf.columns)

23

In [16]:
ratingsdf=spark.read.csv('ratings.csv',header=True,inferSchema=True)

In [17]:
ratingsdf.count()

981756

In [18]:
ratingsdf.printSchema()

root
 |-- book_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- rating: integer (nullable = true)



In [19]:
ratingsdf.first()

Row(book_id=1, user_id=314, rating=5)

In [20]:
ratingsdf.show(10)

+-------+-------+------+
|book_id|user_id|rating|
+-------+-------+------+
|      1|    314|     5|
|      1|    439|     3|
|      1|    588|     5|
|      1|   1169|     4|
|      1|   1185|     4|
|      1|   2077|     4|
|      1|   2487|     4|
|      1|   2900|     5|
|      1|   3662|     4|
|      1|   3922|     5|
+-------+-------+------+
only showing top 10 rows



In [21]:
ratingsdf.select('book_id', 'rating').show(10)

+-------+------+
|book_id|rating|
+-------+------+
|      1|     5|
|      1|     3|
|      1|     5|
|      1|     4|
|      1|     4|
|      1|     4|
|      1|     4|
|      1|     5|
|      1|     4|
|      1|     5|
+-------+------+
only showing top 10 rows



In [22]:
ratingsdf.filter('rating==5').show(10)

+-------+-------+------+
|book_id|user_id|rating|
+-------+-------+------+
|      1|    314|     5|
|      1|    588|     5|
|      1|   2900|     5|
|      1|   3922|     5|
|      1|   5379|     5|
|      1|   5885|     5|
|      1|   6630|     5|
|      1|  10146|     5|
|      1|  10610|     5|
|      1|  10944|     5|
+-------+-------+------+
only showing top 10 rows



In [23]:
ratingsdf.filter('rating==5').select('book_id').show(10)

+-------+
|book_id|
+-------+
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
|      1|
+-------+
only showing top 10 rows



In [27]:
uniqueUsers = ratingsdf.select('user_id').distinct().count()
uniqueUsers

53424

In [28]:
ratingsdf.describe('rating','book_id').show()

+-------+------------------+-----------------+
|summary|            rating|          book_id|
+-------+------------------+-----------------+
|  count|            981756|           981756|
|   mean|3.8565335989797873|4943.275635697668|
| stddev|0.9839408559619973|2873.207414896143|
|    min|                 1|                1|
|    max|                 5|            10000|
+-------+------------------+-----------------+



In [30]:
ratingsdf.count()

981756

In [31]:
ratingsdf.dropDuplicates().count()

980112

In [32]:
# Drow all rows with null values
ratingsdf.dropna().count()

981756

In [33]:
ratingsdf.dropna('any').count()

981756

In [34]:
ratingsdf.dropna('all').count()

981756

In [37]:
ratingsdf.agg({
    'rating': 'max',
    'book_id':'max'
}).show()

+-----------+------------+
|max(rating)|max(book_id)|
+-----------+------------+
|          5|       10000|
+-----------+------------+



In [41]:
ratingsdf.groupBy('rating').count().toPandas().sort_values('rating')

,rating,count
0,1,19575
4,2,63231
1,3,248623
3,4,357366
2,5,292961


In [43]:
ratingsdf.groupBy('rating').count().sort('rating').show()

+------+------+
|rating| count|
+------+------+
|     1| 19575|
|     2| 63231|
|     3|248623|
|     4|357366|
|     5|292961|
+------+------+



In [47]:
booksdf.join(ratingsdf, booksdf.book_id==ratingsdf.book_id).select('user_id','title').show()

+-------+--------------------+
|user_id|               title|
+-------+--------------------+
|    314|Harry Potter and ...|
|    439|Harry Potter and ...|
|    588|Harry Potter and ...|
|   1169|Harry Potter and ...|
|   1185|Harry Potter and ...|
|   2077|Harry Potter and ...|
|   2487|Harry Potter and ...|
|   2900|Harry Potter and ...|
|   3662|Harry Potter and ...|
|   3922|Harry Potter and ...|
|   5379|Harry Potter and ...|
|   5461|Harry Potter and ...|
|   5885|Harry Potter and ...|
|   6630|Harry Potter and ...|
|   7563|Harry Potter and ...|
|   9246|Harry Potter and ...|
|  10140|Harry Potter and ...|
|  10146|Harry Potter and ...|
|  10246|Harry Potter and ...|
|  10335|Harry Potter and ...|
+-------+--------------------+
only showing top 20 rows



In [48]:
ratingsdf.orderBy('rating').show()

+-------+-------+------+
|book_id|user_id|rating|
+-------+-------+------+
|   6689|  18024|     1|
|   6700|  37752|     1|
|   6690|  21836|     1|
|   6687|  12532|     1|
|   6690|  35762|     1|
|   6687|  43545|     1|
|   6690|  43186|     1|
|   6688|  17045|     1|
|   6691|   9116|     1|
|   6693|  49577|     1|
|   6699|  47419|     1|
|   6695|  16829|     1|
|   6687|  13376|     1|
|   6696|   5255|     1|
|   6691|  13845|     1|
|   6696|   8997|     1|
|   6691|  35183|     1|
|   6696|  10492|     1|
|   6691|  46986|     1|
|   6696|  37082|     1|
+-------+-------+------+
only showing top 20 rows



In [49]:
ratingsdf.orderBy(ratingsdf.rating.desc()).show(5)

+-------+-------+------+
|book_id|user_id|rating|
+-------+-------+------+
|   6685|   6962|     5|
|   6685|  18897|     5|
|   6685|   7629|     5|
|   6684|  50457|     5|
|   6685|  13026|     5|
+-------+-------+------+
only showing top 5 rows



In [50]:
ratingsdf.orderBy('rating','book_id').show(5)

+-------+-------+------+
|book_id|user_id|rating|
+-------+-------+------+
|      1|  51480|     1|
|      1|   9246|     1|
|      2|  48687|     1|
|      2|  13794|     1|
|      2|   6063|     1|
+-------+-------+------+
only showing top 5 rows



In [53]:
# Modify existing Columns
ratingsdf.withColumn('rating',ratingsdf.rating*10).show()

+-------+-------+------+
|book_id|user_id|rating|
+-------+-------+------+
|      1|    314|    50|
|      1|    439|    30|
|      1|    588|    50|
|      1|   1169|    40|
|      1|   1185|    40|
|      1|   2077|    40|
|      1|   2487|    40|
|      1|   2900|    50|
|      1|   3662|    40|
|      1|   3922|    50|
|      1|   5379|    50|
|      1|   5461|    30|
|      1|   5885|    50|
|      1|   6630|    50|
|      1|   7563|    30|
|      1|   9246|    10|
|      1|  10140|    40|
|      1|  10146|    50|
|      1|  10246|    40|
|      1|  10335|    40|
+-------+-------+------+
only showing top 20 rows



In [54]:
# Add new Columns
ratingsdf.withColumn('ratingTen',ratingsdf.rating*10).show()

+-------+-------+------+---------+
|book_id|user_id|rating|ratingTen|
+-------+-------+------+---------+
|      1|    314|     5|       50|
|      1|    439|     3|       30|
|      1|    588|     5|       50|
|      1|   1169|     4|       40|
|      1|   1185|     4|       40|
|      1|   2077|     4|       40|
|      1|   2487|     4|       40|
|      1|   2900|     5|       50|
|      1|   3662|     4|       40|
|      1|   3922|     5|       50|
|      1|   5379|     5|       50|
|      1|   5461|     3|       30|
|      1|   5885|     5|       50|
|      1|   6630|     5|       50|
|      1|   7563|     3|       30|
|      1|   9246|     1|       10|
|      1|  10140|     4|       40|
|      1|  10146|     5|       50|
|      1|  10246|     4|       40|
|      1|  10335|     4|       40|
+-------+-------+------+---------+
only showing top 20 rows



In [56]:
# drop a column
ratingsdf.drop('rating').show(5)

+-------+-------+
|book_id|user_id|
+-------+-------+
|      1|    314|
|      1|    439|
|      1|    588|
|      1|   1169|
|      1|   1185|
+-------+-------+
only showing top 5 rows

